<h1 style="color:blue">Praktikum 10</h1>

<h3 style="color:blue">Veebiga suhtlemine ja tekstiandmete kogumine veebist</h3>

Tänases praktikumis õpime, kuidas luua programme, mis suhtlevad veebiga: teevad veebis päringuid ja korjavad sealt tekstiandmeid. Alustame lihtsamatest programmidest, mis "tõmbavad alla" ühe veebilehe sisu ja otsivad sealt meid huvitavat infot, ning lõpetame keerulisemate programmidega -- nn "veebiämblikega", mis liiguvad andmeid korjates veebilehelt veebilehele.

## Veebiga suhtlemine

Igapäevaselt saame veebiga suhelda internetilehitseja ehk brauseri vahendusel, külastades uudisteportaale, blogisid ning teisi lehekülgi. Kui tahame programmaatiliselt veebist tekste alla laadida, siis peame natukene tutvuma ka tehnilise poolega: millised päringud milliste argumentidega seal taustal toimuvad. Sellest tulebki järgnevalt juttu.

#### _Get_ päringud

Kui sisestame aadressi brauseri aadressilahtrisse, teeb brauser tavaliselt **_get_** päringu kindlale ressursile, mille tagajärjel saadetakse dokument serverist meie arvutisse. Andmete sisestamine / saatmine veebilehe kaudu (täidame lehel oleva lahtri või vormi ja vajutame ENTER'it / "nuppu SAADA" vms) toimub samuti kas _get_ või _post_ päringute kaudu. _Get_-päringuid kasutatakse tavaliselt siis, kui saata tuleb väike hulk andmeid (kuna _get_ päringul on suuruse limiit), ning _post_-päringud on reserveeritud suurtes kogustes andmete saatmiseks.

_Get_-päringutega on lihtne selles mõttes, et standardi kohaselt on kogu informatiivne teave serverile talletatud päringu kehas ning seega ressursi pärimine pole oluliselt keerulisem brausimisest. Lihtsustatud näide guugeldamisest. Kui lähed lehele [www.google.ee](https://www.google.ee) ning sisestad otsinguaknasse "keeletehnoloogia", siis peaks brauser minema automaatselt uuele leheküljele***, mille veebiaadress on:

    https://www.google.ee/search?q=keeletehnoloogia 

(*** -- tõsi, praktikas võib _google_-otsingu aadress olla keerulisem, sõltuvalt sellest, millised on olnud eelmised otsingud / _google_ kasutamise  ajalugu)

Sellisel kujul olev veebiaadress tähendabki  **parameetritega _get_-päringu** tegemist: leheküljele `https://www.google.ee/search` tehakse päring, millega pannakse kaasa parameeter `q=keeletehnoloogia`. Server, mis selle päringu vastu võtab, sooritab vastava otsingu (leiab veebilehed, mis mainivad _keeletehnoloogiat_) ja tagastab otsingu tulemusi sisaldava veebilehe. Ja tegelikult me ei peagi päringu tegemiseks alustama [www.google.ee](https://www.google.ee) leheküljest -- kui meil on teada _get_-päringule vastav veebiaadress (ja me oleme välja selgitanud, mida üks või teine parameeter teeb), siis võime ka käsitsi konstrueerida vastava päringu ja sisestada selle brauseri kaudu. Proovi käsitsi muuta _google_ päringu veebiaadressi nii, et otsitakse mingit teist märksõna.

Milleks see info vajalik on? Kui uurid veebilehtede lähtekoodi ja leiad sealt lingid, mis näevad välja nagu parameetritega _get_-päringud, siis kõnelevad need üht-teist selle kohta, kuidas tuleb serveriga suhelda. Näiteks, kui on tegemist otsinguportaaliga, mis tagastab mitu lehekülge tulemusi, on päringuparameetrites sageli kirjas, mitmendalt leheküljelt alates tuleb (uues päringus) tulemusi kuvada. Selle teadmise abil saad luua programmi, mis teeb otsingumootoriga päringu, sirvib päringu tulemusi üle mitme lehekülje ja korjab andmeid.

Lisaks _get_-päringutele võib server vastu võtta ka _post_-päringuid. Et _post_-päringute konstrueerimine on sageli keerulisem kui _get_-päringute konstrueerimine (ainuüksi veebilehe lähtekoodi vaadates ei pruugi me saada täit ülevaadet päringuks vajaminevast), siis me selle teemaga süvitsi ei lähe. Praktikas võime aga juba päris palju ära teha ka _get_-päringutega.

### Programmaatiline suhtlus

#### Päringud teegi `urllib.request` abil

Pythoni standardteegis vastutab veebiga suhtlemise eest teek `urllib.request`.

In [1]:
from urllib.request import urlopen

url = 'http://www.folklore.ee/rl/folkte/myte/kalev/56.html'
response = urlopen(url)

Funktsioon `urlopen` sooritab päringu etteantud aadressile. Tulemuseks saame vastuse serverilt, mis on [_HTTPResponse_ objekti](https://docs.python.org/3/library/http.client.html#httpresponse-objects) kujul.

Vastuse päises (_header_) on toodud päringu kohta mitmesuguseid metaandmeid. Nt leiab sealt päringu tegemise aja (`Date`), informatsiooni serveri kohta (`Server`), ressurssi viimase muutmiskuupäeva (`Last-Modified`) jms:

In [2]:
response.getheaders()

[('Date', 'Tue, 01 Nov 2022 09:13:29 GMT'),
 ('Server',
  'Apache/2.4.53 (FreeBSD) OpenSSL/1.1.1o-freebsd mod_auth_gssapi/1.6.3 PHP/7.4.29'),
 ('Last-Modified', 'Wed, 23 Jan 2002 11:14:01 GMT'),
 ('ETag', '"4357-398366c31f040"'),
 ('Accept-Ranges', 'bytes'),
 ('Content-Length', '17239'),
 ('Connection', 'close'),
 ('Content-Type', 'text/html')]

Kui korjame veebist tekstiandmeid, on meie jaoks olulisim päise andmeväli `Content-Type`, mis ütleb, mis liiki ressurssiga on tegu:

In [3]:
response.getheader('Content-Type')

'text/html'

Lisaks võib `Content-Type` sisaldada ka dokumendi kodeeringut (nt `Content-Type: text/html; charset=utf-8` ).

Muude päise andmeväljade kohta vt lähemalt https://en.wikipedia.org/wiki/List_of_HTTP_header_fields .

Vastuse objektist saab kätte ka päringu staatuskoodi. Sisuliselt annab see lühiteavet päringu õnnestumise kohta: kas päring õnnestus (kood: 200), päritud ressursile puudus õigus (401), ressurssi ei leidunud (404), server läks katki päringu peale (500) või muud sellist. 

In [4]:
response.status

200

Võimalike staatuskoodide kohta vt lähemalt https://en.wikipedia.org/wiki/List_of_HTTP_status_codes .

Kõige oluliseim on aga lehe sisu kätte saada. 

Sarnaselt failiviitadega sisaldab vastuse objekt funktsiooni `read()`, mis tagastab vastuse sisu (nn vastuse keha) sõne kujul. Erinevalt faili avamisest on meil aga tagastatud sisu endiselt kodeeritud kujul, seega tuleb enne kasutamist dekodeerida:

In [5]:
content = urlopen(url).read().decode('utf8') # proovime dekodeerida utf8 abil, kuivõrd kodeeringut pole 'Content-Type' päises

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf5 in position 4372: invalid start byte

Lugemisel tuli kodeeringuviga: paistab, et tegemist siiski pole `UTF-8`'ga.

Milliseid kodeeringuid me veel teame? Kuna [veebileht ise](http://www.folklore.ee/rl/folkte/myte/kalev/) paistab olevat üksjagu ammu loodud, võiks ehk oletada, et loomisel on kasutatud Windows'i masinat. Seega võime katsetada Windows'i tüüpilist kodeeringut:

In [6]:
content = urlopen(url).read().decode('Windows-1257')
content

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 TRANSITIONAL//EN">\r\n\r\n<HTML>\r\n\r\n<HEAD>\r\n<TITLE>Kalevipoeg - Kalevipoja varandus</TITLE>\r\n<META NAME="GENERATOR" CONTENT="Arachnophilia 3.9">\r\n<META NAME="FORMATTER" CONTENT="Arachnophilia 3.9">\r\n</HEAD>\r\n\r\n<BODY BGCOLOR="#003050" TEXT="#AAAAAA" LINK="#AFAFAF" VLINK="#8F8F8F" ALINK="#FFFF00" BGPROPERTIES=FIXED>\n\r\n<CENTER><FONT COLOR="#CFCFCF"><H1><B>381. KALEVIPOJA VARANDUS</B></H1></FONT></CENTER>\r\n[Kalevipoja varandus T&uuml;rsam&auml;el]<BR>\r\n&Auml;rina metsas Porkuni raja peal on kaks suurt kivi, mida &uuml;hte nendest Viru &auml;mmaks kutsutakse, muidu ka veel suuremat teumeeste &auml;mmaks, v&auml;hemat teuvaimude &auml;iaks. Viru &auml;mma all pidada nimelt Kalevipoja terve varandus ja tema t&ouml;&ouml;riistad varjul olema. Kalevipoja hobuse kondid ja hobuseriistad pidada aga Assamalla luhas olema. 1909. aasta suvel k&auml;idud Soomest &uuml;ht T&uuml;rsam&auml;ge V&auml;ike-Maarja kihelkonnas taga otsimas. 

Lehe sisu uurides näeme, et ehkki kodeeringule saime teisel korral pihta, on seal endiselt omajagu veidraid sümboleid. HTML-lehekülgedel (ja eriti vanematel lehekülgedel, mis ei kasuta `UTF-8`'t) kasutatakse sageli [ASCII tabelist](https://www.rapidtables.com/code/text/ascii-table.html) väljajäävate sümbolite _escape_'imist ehk konverteerimist HTML olemiteks. Näiteks on 'ä' asendatud olemiga _& auml;_ ('a' _umlaut_). Kui soovime sisu uuesti loetavaks teha, peame olemid tagasi sümboliteks konverteerima ehk _unescape_'ima.

Selleks on mitu võimalust. Üks võimalus on rakendada funktsiooni `html.unescape`:

In [7]:
import html
html.unescape(content)

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 TRANSITIONAL//EN">\r\n\r\n<HTML>\r\n\r\n<HEAD>\r\n<TITLE>Kalevipoeg - Kalevipoja varandus</TITLE>\r\n<META NAME="GENERATOR" CONTENT="Arachnophilia 3.9">\r\n<META NAME="FORMATTER" CONTENT="Arachnophilia 3.9">\r\n</HEAD>\r\n\r\n<BODY BGCOLOR="#003050" TEXT="#AAAAAA" LINK="#AFAFAF" VLINK="#8F8F8F" ALINK="#FFFF00" BGPROPERTIES=FIXED>\n\r\n<CENTER><FONT COLOR="#CFCFCF"><H1><B>381. KALEVIPOJA VARANDUS</B></H1></FONT></CENTER>\r\n[Kalevipoja varandus Türsamäel]<BR>\r\nÄrina metsas Porkuni raja peal on kaks suurt kivi, mida ühte nendest Viru ämmaks kutsutakse, muidu ka veel suuremat teumeeste ämmaks, vähemat teuvaimude äiaks. Viru ämma all pidada nimelt Kalevipoja terve varandus ja tema tööriistad varjul olema. Kalevipoja hobuse kondid ja hobuseriistad pidada aga Assamalla luhas olema. 1909. aasta suvel käidud Soomest üht Türsamäge Väike-Maarja kihelkonnas taga otsimas. Seal mäe peal peab üks suur vöötidega kivi olema ja kolm munasuurust auku seal k

Teine võimalus on kasutada HTML sisu lugemiseks moodulit _BeautifulSoup_, mis teeb teisendused meie eest:

In [8]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(content, 'html.parser')

print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 TRANSITIONAL//EN">
<html>
 <head>
  <title>
   Kalevipoeg - Kalevipoja varandus
  </title>
  <meta content="Arachnophilia 3.9" name="GENERATOR"/>
  <meta content="Arachnophilia 3.9" name="FORMATTER"/>
 </head>
 <body alink="#FFFF00" bgcolor="#003050" bgproperties="FIXED" link="#AFAFAF" text="#AAAAAA" vlink="#8F8F8F">
  <center>
   <font color="#CFCFCF">
    <h1>
     <b>
      381. KALEVIPOJA VARANDUS
     </b>
    </h1>
   </font>
  </center>
  [Kalevipoja varandus Türsamäel]
  <br/>
  Ärina metsas Porkuni raja peal on kaks suurt kivi, mida ühte nendest Viru ämmaks kutsutakse, muidu ka veel suuremat teumeeste ämmaks, vähemat teuvaimude äiaks. Viru ämma all pidada nimelt Kalevipoja terve varandus ja tema tööriistad varjul olema. Kalevipoja hobuse kondid ja hobuseriistad pidada aga Assamalla luhas olema. 1909. aasta suvel käidud Soomest üht Türsamäge Väike-Maarja kihelkonnas taga otsimas. Seal mäe peal peab üks suur vöötidega kivi olema ja kol

#### Päringud teegi `requests` abil 

Alternatiiviks `urllib.request`-ile on teek `requests`. Paraku `requests` standardteekide hulka ei kuulu, seega tuleb installida ( installikäsk: `conda install requests` ). Nagu teegi nimigi ([Requests: HTTP for Humans](https://requests.readthedocs.io/en/latest/)) ütleb, on tegemist veebipäringute teegiga, mille liides on standardteegi omast veidi lihtsam ja inimlikum.

In [9]:
import requests

response = requests.get(url)  # teeme päringu

`Requests`'i vastus võimaldab kõik vajaliku vastuse kohta kätte saada atribuutide ja meetodite kaudu.

In [10]:
print('Status:',response.status_code)
print('Content-Type:',response.headers['content-type'])
print('Encoding:',response.encoding)
print('Content:',response.text[:20])

Status: 200
Content-Type: text/html
Encoding: ISO-8859-1
Content: <!DOCTYPE HTML PUBLI


Päringu vastuse sisuline osa on atribuudi `text` all. Võime täheldada, et ka teksti kodeeringu on teek juba ise välja nuputanud, seega ei pea me käsitsi hakkama eri kodeeringuid katsetama. Küll aga ei pääse me lehel olevate HTML olemitega jändamisest: need tuleb endiselt ise sümboliteks tagasi konverteerida.

### Ülesanne 1. "Uusimad uudised" (1 p)

Alati ei tulegi veebist info kättesaamiseks HTML-i parsida. Veebis toimetavate otsingumootorite ja veebiämblike jaoks pakuvad veebiportaalid sageli välja nn sisukaarte (ingl _site map_), millelt leiab otseviited olulistele ja päevakohastele lehekülgedele. Nt Postimehe sisukaardilt ( https://www.postimees.ee/sitemap ) leiame viited kolmele alamkaardile: värskete uudiste loendile, aktuaalsete teemade loendile ning võtmesõnade loendile. Kui soovime saada kätte värskete uudiste (pealkirjade) nimekirja, siis sisukaardi abil on seda teha üsna mugav:  info on esitatud kompaktsel XML kujul ning me ei pea nägema vaeva selle ülesotsimisega HTML lehe sisu osast.

Järgnevalt ongi ülesandeks kirjutada programm, mis teeb päringu Postimehe uudiste sisukaardile ( https://www.postimees.ee/sitemap/news ), nopib sealt välja artiklite pealkirjad, veebiaadressid ja viimase muutmise kuupäevad (`lastmod`) ning väljastab korjatud info muutmiskuupäevade järgi sorteeritult.

**Detaile:**
   - Kasuta päringu tegemiseks teeki `urllib.request` või `requests`;
   - Kuna tagastatav sisu on XML, siis selle parsimiseks on ilmselt kõige mugavam kasutada _BeautifulSoup_'i. Uuri, millise struktuuriga on tagastatav XML dokument ja kasuta _BeautifulSoup_-i selle sisust [alamosade otsimiseks](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree). Kui artikli pealkirjade kättesaamine eriti ei õnnestu, siis abi võib saada [siit](https://stackoverflow.com/questions/3058912/how-can-i-access-namespaced-xml-elements-using-beautifulsoup);
   
Sisukaartide kohta pakub rohkem infot lehekülg _sitemaps.org_, vt nt https://www.sitemaps.org/index.html;

### Viisakusreeglid veebisuhtluses

Eelmises ülesandes toimetasid ühe veebiressurssi piires ja tõenäoliselt tegid ainult väikese arvu päringuid. Kui aga päringute arv kasvab suureks või päringuid tehakse paljudele veebilehtedele, muutub oluliseks veebisuhtluse viisakusreeglite järgimine.

#### Päringute tegemise kiirus

Programm suudab päringuid teha märksa kiiremini kui (veebilehitsejat kasutav) inimene. Liiga kiire / sage lehtede tõmbamine tähendab aga seda, et server peab palju aega kulutama programmi päringute teenindamiseks ning teiste kasutajate päringute teenindamine võib saada häiritud. Halvimal juhul võib liiga kiire lehtede tõmbamine ka serveri kokku jooksutada.

Seega üks oluline viisakusreegel on limiteerida oma päringuid ajaliselt: lisada kahe päringu tegemise vahele ajavahe. Soovitav on ühelt domeeniaadressilt mitte tõmmata sagedamini kui iga 10 sekundi tagant.

In [11]:
import time
time.sleep(10)   # ootame 10 sek

#### Veebilehe enda juhised robotitele (`robots.txt`)

Veebilehtede omanikud panevad sageli ise üles juhiseid selle kohta, mida lehel ringiluusivad "robotid" võivad teha ja mida mitte. Tavakohaselt on need juhised kättesaadaval failis nimega `robots.txt`. Näiteks Postimehe saidi vastav fail on https://www.postimees.ee/robots.txt 

Üks näide `robots.txt` sisu kohta koos lühiselgitustega:

    User-agent: *                               # järgnevad juhised kehtivad kõigile robotitele
    Disallow: /secret                           # keela alamleht
    Disallow: /secret/*                         # keela alamlehed
    User-agent: crawler123                      # järgnevad juhised kehtivad ainult robotile nimega 'crawler123'
    Allow:    /secret/*                         # luba alamlehed
    User-agent: *                               # järgnevad juhised kehtivad kõigile robotitele
    Crawl-delay: 25                             # server nõuab 25-sekundilist lehtede tõmbamise vahet
    Sitemap: http://www.domain.ext/sitemap      # sisukaardi asukoht

Milline on faili `robots.txt` formaat, sellest on täpsemalt juttu [siin](http://www.robotstxt.org/robotstxt.html).

#### Veebiämbliku / roboti identiteedi määramine

HTTP päringute päises on ka info selle kohta, kes või mis on päringuid tegev agent. Kui agent ja selle eesmärgid on täpsustamata, võib see äratada kahtlusi, kuna identiteedita luusib sageli ringi pahavara. Seega on väga soovitav määrata oma programmis ka päringu tegija identiteet. Vaatame teegi `requests` näitel, kuidas seda teha.

In [12]:
import requests
# Uurime vaikimisi kasutatavat päringupäist: mis on agendi nimi?
headers = requests.utils.default_headers()
headers

{'User-Agent': 'python-requests/2.28.1', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}

Agendi nimi oli mittemidagiütlev (`'python-requests/X.XX.X'`). Kirjutame sinna midagi täpsemat, näiteks:

In [13]:
headers.update(
  { 'User-Agent': 'my crawler (minu.email@ut.ee) / on studying purpose', }
)

Päringuid tehes paneme nüüd alati kaasa ka uuendatud päise:

In [14]:
response = requests.get('http://httpbin.org/user-agent', headers=headers)

In [15]:
response.text

'{\n  "user-agent": "my crawler (minu.email@ut.ee) / on studying purpose"\n}\n'

Lehekülg [http://httpbin.org](http://httpbin.org), millele äsja päringu tegime, sobib hästi veebipäringute testimiseks: sisuliselt nopib see välja mingi alamosa meie päringust ja saadab meile tagasi. Antud juhul siis nopiti päisest välja andmed `'user-agent'` kohta ja saadeti meile tagasi JSON kujul. Seega saame veenduda, et paistame veebist välja tõesti sellise identiteediga, nagu eelnevalt määrasime.

Näide päringu päise uuendamise kohta pärineb [sellest allikast](https://stackoverflow.com/a/10606260), huvi korral leiad sealt ka näpunäiteid, kuidas seda veelgi mugavamalt teha.

##### Eksimine viisakusreeglite vastu

Veebisuhtluse viisakusreeglite vastu eksimine on ebaeetiline, lühema/väiksema rikkumise korral aga üldiselt tagajärgedeta. Pikema rikkumise korral on võimalik ühendusevõtmine vastava lehe süsteemi administraatori poolt ja IP aadressi blokeerimine. Halval juhul võib see lõppeda ka kohtuasjaga.

### Ülesanne 2. Kalevipoega külastamas (2 p)
Kasuta `requests` teeki [Kalevipoja Vikipeedia veebilehe](https://et.wikipedia.org/wiki/Kalevipoeg) sisu allalaadimiseks ning leia _BeautifulSoup_'i abil kõik (paragrahvi märgendite vahel olevad) lingid. Järgne linkide abil viidatud lehtedele ning leia  kõikide illustratiivsete piltide aadressid. 

**Detaile:**
  - Programm peab jälgima veebisuhtluse viisakusreegleid: päringu tegija identiteet peab olema määratud ning päringute tegemise vahel peavad olema pausid;
  - Kui veebilehe sisu on käes, siis esimese sammuna [leia](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#searching-the-tree) Kalevipoja veebilehelt `<p>`-märgendite vahel olevad lingid (`<a>`-märgendid) ning võta välja linkide atribuutide `href` väärtused. Kuidas saab BeautifulSoup'i abil atribuutide väärtused kätte saada, sellest räägitakse [siin](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#attributes);
  - Atribuutide `href` väärtused sisaldavad suhtelisi aadresse (nt `'/wiki/Allikas'` ja `'/wiki/Kivi'`), uute päringute tegemiseks pead nende põhjal aga looma absoluutaadressid (lisage nende ette Vikipeedia  baas-aadress);
  - Pildid saad kätte `<img>` märgendeid otsides. Siiski ei esinda kõik `<img>` märgendid illustratiivseid pilte -- paljud sisaldavad kujundamisgraafikat (nt _svg_-d). Eesmärgiks on aga leida just illustratiivsed pildid, vaatluse alt jäävad välja Vikipeedia logo ja kujunduselemendid. Selleks, et "eraldada terad sõkaldest", uuri `<img>` märgendite atribuute ning püüa leida, mis on kõigil illustratiivsetel piltidel ühist;
  - Töö tulemusena kuvab skript loendi külastatud lehtedest ning iga lehe all loendi sealt leitud illustratiivsete piltide aadressidest. Piltide aadressid on sellisel kujul, et veebibrauseris vastavale aadressile minnes avanebki pilt;


## Veebiämblikud ja _crawl_'imine

Nüüdseks oleme juba salamahti tegelenud mitmete asjadega, mida teevad veebiämblikud ehk _crawler_'id. Oleme tõmmanud üksikuid lehti, valinud nendelt URL-e ja koostanud uute külastatavate lehtede nimekirja ning eraldanud lehtedelt olulisi dokumendiosi. Samas "päris veebiämbliku" ehitamiseni on veel üksjagu maad minna. 

Kui on plaanis laiaulatuslikum tekside korjamine veebist, siis praktikas polegi otstarbekas hakata nullist veebiämblikku ehitama, vaid on soovitav õppida kasutama valmis raamistikke. 

Käesoleva praktikumi viimase ülesande lahendamisel saad katsetada raamistikku [_scrapy_](https://scrapy.org/), mis koondab endas kokku mitmed veebist "andmete kokkukraapimise" (ingl [_web scraping_](https://en.wikipedia.org/wiki/Web_scraping)) parimad praktikad. _Scrapy_'s on võimalik kasutada nii valmis "kraapijaid" kui ka kirjutada enda omi. Raamistik võimaldab muuta praktiliselt kõike alates lehelt informatsiooni eraldamisest kuni URL-ide valimise ja tõmbamissageduseni. Lisaks on võimalik _scrapy_ _crawler_'i panna jooksma [_Scrapy Cloud_](https://www.zyte.com/scrapy-cloud/) keskkonnas, milles lubatakse tasuta jooksutada piiramatult kraapijaid (kuigi tasuta versioonil on ka omad piirangud, nt lühike kraapimise ning andmete säilitamise aeg). 

### Ülesanne 3. Tsitaadikoguja (2,5 p)

Loo veebiämblik, mis korjab leheküljelt http://tsitaadid.ee/ 1000 unikaalset tsitaati. Iga tsitaadi kohta moodustatakse andmekirje, mis sisaldab tsitaadi autorit, teemat ja teksti. Tsitaadid salvestatakse faili, igal real ühe tsitaadi andmed JSON formaadis (täpsemalt siis: fail on [_JSON Lines_ formaadis](http://jsonlines.org/)). 
Kasuta tsitaatide korjamiseks raamistikku [_scrapy_](https://scrapy.org/). 
Lahendusena esita loodud programm (kogu `scrapy` projekt) ja kogutud tsitaatide fail.

**Üldised nõuded**:
  * Kuigi lehekülg http://tsitaadid.ee/ väljastab tsitaate juhuslikult, siis ei piisa lahendusest, mis koguaeg külastabki ainult seda lehekülge; programm peab tsitaate korjates liikuma leheküljelt leheküljele, seega pead iga lehe külastamisel üles otsima nupu/lingi, mis viib järgmisele lehele;
  * Kuna http://tsitaadid.ee/ väljastab tsitaate juhuslikult, siis võib nende seas olla ka korduvaid; programm peaks korduseid vältima ja korjama ainult unikaalseid tsitaate;
  * **(!)** Ülesande lahendus peab järgima veebisuhtluse viisakusreegleid: ei tohi serverit sagedaste päringutega üle uputada (päringute vahe peaks olema vähemalt 10 sek) ning päringu päises tuleks täpsustada oma veebiämbliku nimi / eesmärk ja kontakt.

**Täpsemalt [_scrapy_](https://scrapy.org/) kohta:**
   - _scrapy_ pole standardteek ning tuleb seetõttu juurde [installida](https://docs.scrapy.org/en/latest/intro/install.html);
   - et lahendada ülesanne _scrapy_-i abil, selleks tuleb natukene lisaks õppida: kuidas _scrapy_-i abil veebilehe lähtekoodist andmeid õngitseda, kuidas veebiämblik luua ja kuidas seda käsurealt jooksutada;
   - kaustas `scrapy_tutorial` on _scrapy_-i projekti mall, mille võid veebiämbliku loomisel aluseks võtta. Sisuliselt tuleks oma kood kirjutada skripti `scrapy_tutorial\tutorial\spiders\quotes_spider.py`. Mall on loodud  [selle abimaterjali](https://docs.scrapy.org/en/latest/intro/tutorial.html) järgi ning viidatud abimaterjal peaks ühtlasi olema ka väga heaks eeskujuks (vihje: seal lahendatakse sarnast ülesannet ingliskeelsetel andmetel);
